安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

# 指令微调

In [1]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets

d:\ana\envs\machinelearn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@dataclass
class ModelArguments:
    """Arguments for model."""
    model_name_or_path: str = field(
        default="/kaggle/input/Qwen-2.5-0.5B",
        metadata={"help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."}
    )
    torch_dtype: str = field(
        default="bfloat16",
        metadata={
            "help": "The precision to use when loading the model.",
            "choices": ["bfloat16", "float16", "float32"]
        },
    )

@dataclass
class DataArguments:
    """Arguments for data."""
    dataset_path: str = field(
        default="alpaca-cleaned",
        metadata={"help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."}
    )
    max_seq_length: int = field(
        default=1024,
        metadata={"help": "The maximum sequence length for the input data."}
    )





In [3]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()


    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        torch_dtype=model_args.torch_dtype,
        device_map="auto",
    )
    
    model.gradient_checkpointing_enable()
    print(model.device)

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset(data_args.dataset_path)

    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    
    def data_collator(batch: List[Dict[str, str]]) -> Dict[str, torch.Tensor]:
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        instructions = [sample['instruction'] for sample in batch]
        inputs = [sample['input'] for sample in batch]
        outputs = [sample['output'] for sample in batch]
        input_texts = [f"{instruction} {input} {output}" for instruction, input, output in zip(instructions, inputs, outputs)]
        tokenized_inputs = tokenizer(
        input_texts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=tokenizer.model_max_length
        )
        labels = tokenized_inputs["input_ids"].clone()
        for i, feature in enumerate(batch):
            instruction_length = len(tokenizer(feature['instruction'], add_special_tokens=False)["input_ids"])
             # 检查 input 是否为空字符串
            if feature['input'] == None:
                feature['input'] = ""
            input_length = len(tokenizer(feature['input'], add_special_tokens=False)["input_ids"])
            output_start = instruction_length + input_length - 1

            # Mask everything except the output part with -100
            labels[i, :output_start] = -100
            # 将padding部分的token也设置为-100
            labels[labels == tokenizer.pad_token_id] = -100
        tokenized_inputs["labels"] = labels
        
        return tokenized_inputs

    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
    )



    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset["train"],
        tokenizer=tokenizer,
        
    )

    # Step 6: Train!
    trainer.train()

In [ ]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    "--model_name_or_path", "qwen2.5",
    "--dataset_path", "Alpaca Cleaned",
    "--output_dir", "working/lr_1e-6_bs_1",
    "--learning_rate", "1e-6",
    "--num_train_epochs", "2",
    "--per_device_train_batch_size", "1",  # 每个设备的训练批次大小
    "--save_steps", "500",
    "--save_total_limit", "1",
    "--remove_unused_columns", "False",
]

finetune()

# 评测模型

In [ ]:
PLM_MODEL_PATH = "qwen2.5"
SFT_MODEL_PATH = "working/lr_1e-6_bs_1/checkpoint-103520"

如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测。

In [ ]:
!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {PLM_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "working/evals/plm" \
    --debug

In [ ]:
!opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {SFT_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 2 \
    --work-dir "working/evals/sft" \
    --debug